In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


# Load Datas

In [3]:
df = pd.read_csv('sdn_ml.csv')
df.index = df['date']
df.drop('date', axis=1, inplace=True)
df.head()

,hour,minut,second,day,month,delay,sensor_id,temperature,label
date,,,,,,,,,
2004-03-31 03:38:15.757551,3,38,15,31,3,-2.774340e+06,1,122.153,1
2004-02-28 00:59:16.027850,0,59,16,28,2,2.403061e+02,1,199.884,1
2004-02-28 01:03:16.333930,1,3,16,28,2,1.796795e+02,1,193.024,0
2004-02-28 01:06:16.013453,1,6,16,28,2,3.076463e+01,1,191.652,1
2004-02-28 01:06:46.778088,1,6,46,28,2,1.192144e+02,1,19.175,1


# Separing dataset

In [4]:
train_size = int(len(df) * 0.9)
test_size = len(df) - train_size
train, test = df.iloc[0:train_size], df.iloc[train_size:len(df)]


# Normalizing

In [5]:
from sklearn.preprocessing import RobustScaler

In [6]:
f_columns = ['delay','temperature']

f_transformer = RobustScaler()
lb_transformer = RobustScaler()

f_transformer = f_transformer.fit(train[f_columns].to_numpy())
train.loc[:,f_columns] = f_transformer.transform(train[f_columns].to_numpy())

#lb_transformer = lb_transformer.fit(train[['label']])
#train['label'] = lb_transformer.transform(train[['label']])

C:\Users\silva\anaconda3\lib\site-packages\pandas\core\indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


In [1]:
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i: (i+time_steps)].to_numpy()
        Xs.append(v)
        ys.append(y.iloc[i+time_steps])
    return np.array(Xs), np.array(ys)

In [2]:
TIME_STEPS = 30

X_train,Y_train = create_dataset(train, train.label, time_steps=TIME_STEPS)
X_test,Y_test = create_dataset(test, test.label, time_steps=TIME_STEPS)

NameError: name 'train' is not defined

In [9]:
#deoeting label
train.drop('label', axis=1, inplace=True)


C:\Users\silva\anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [10]:
Y_train

array([0, 1, 0, ..., 0, 0, 1], dtype=int64)

In [11]:
print(X_train.shape, Y_train.shape)

(80934, 30, 9) (80934,)


# Training

In [12]:
model = keras.Sequential()
model.add(
    keras.layers.Bidirectional(
        keras.layers.LSTM(
            units=128,
            input_shape=(X_train.shape[1],X_train.shape[2])
        )
    ))
model.add(keras.layers.Dense(units=128))
model.add(keras.layers.Dense(units=128))
model.add(keras.layers.Dense(units=128))
model.add(keras.layers.Dropout(rate=0.2))
model.add(keras.layers.Dense(units=1))
#model.add(tf.keras.layers.ThresholdedReLU(theta=1))

# Compilation

In [13]:
loss ="mse"
optim = tf.keras.optimizers.SGD(
    learning_rate=0.001)
metrics=["accuracy"]

In [14]:
model.compile(loss=loss, optimizer=optim, metrics=metrics)

In [15]:
history = model.fit(
    X_train, Y_train, 
    epochs=50, 
    batch_size= 64,
    validation_split=0.1,
    shuffle=False
)

Epoch 1/50
1139/1139 [==============================] - 77s 61ms/step - loss: 0.2910 - accuracy: 0.6032 - val_loss: 0.1856 - val_accuracy: 0.6804
Epoch 2/50
1139/1139 [==============================] - 62s 54ms/step - loss: 0.2479 - accuracy: 0.6296 - val_loss: 0.1836 - val_accuracy: 0.6836
Epoch 3/50
1139/1139 [==============================] - 64s 56ms/step - loss: 0.2365 - accuracy: 0.6448 - val_loss: 0.1847 - val_accuracy: 0.6854
Epoch 4/50
1139/1139 [==============================] - 65s 57ms/step - loss: 0.2306 - accuracy: 0.6505 - val_loss: 0.1859 - val_accuracy: 0.6937
Epoch 5/50
1139/1139 [==============================] - 64s 57ms/step - loss: 0.2271 - accuracy: 0.6563 - val_loss: 0.1857 - val_accuracy: 0.6920
Epoch 6/50
1139/1139 [==============================] - 65s 57ms/step - loss: 0.2256 - accuracy: 0.6549 - val_loss: 0.1860 - val_accuracy: 0.6922
Epoch 7/50
1139/1139 [==============================] - 73s 64ms/step - loss: 0.2236 - accuracy: 0.6584 - val_loss: 0.1862 -

KeyboardInterrupt: 

In [ ]:
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='validation')
plt.legend();

In [ ]:
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='validation')
plt.legend();

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
plt.plot(Y_test, marker='.', label='true')
plt.plot(y_pred,'r',marker='.', label='predicted')
plt.legend();

In [ ]:
from sklearn.metrics import confusion_matrix


In [ ]:
#confusion_matrix(y_test_inv, y_pred_inv)

In [ ]:
y_pred

In [ ]:
Y_test

In [ ]:
# y_train_inv = lb_transformer.inverse_transform(Y_train.reshape(1,-1)) 
# y_test_inv = lb_transformer.inverse_transform(Y_test.reshape(1,-1))
# y_pred_inv = lb_transformer.inverse_transform(y_pred)